In [1]:
import pandas as pd
import numpy  as np

from fastapi import FastAPI

import uvicorn

from sklearn.metrics.pairwise        import cosine_similarity
from sklearn.utils.extmath           import randomized_svd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise        import linear_kernel


df = pd.read_csv("data_set/ds_model.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42195 entries, 0 to 42194
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   overview    42195 non-null  object 
 1   popularity  42195 non-null  float64
 2   title       42195 non-null  object 
dtypes: float64(1), object(2)
memory usage: 989.1+ KB


In [3]:
import nltk
from nltk.tokenize import word_tokenize

# Descargar los recursos necesarios de NLTK (ejecutar solo una vez)
nltk.download('punkt')

# Muestra de datos
overview_sample = "Led by Woody, Andy's toys live happily in his ..."

# Tokenización de la muestra
tokens = word_tokenize(overview_sample)

# Imprimir los tokens
print(tokens)

['Led', 'by', 'Woody', ',', 'Andy', "'s", 'toys', 'live', 'happily', 'in', 'his', '...']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Descargar los recursos necesarios de NLTK (ejecutar solo una vez)
nltk.download('punkt')

# Tokenización de la columna "overview"
df['overview_tokens'] = df['overview'].apply(lambda x: word_tokenize(str(x)))

# Tokenización de la columna "title"
df['title_tokens'] = df['title'].apply(lambda x: word_tokenize(str(x)))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Ahora para la columna 'title'

In [5]:
df

,overview,popularity,title,overview_tokens,title_tokens
0,"Led by Woody, Andy's toys live happily in his ...",21.946943,Toy Story,"[Led, by, Woody, ,, Andy, 's, toys, live, happ...","[Toy, Story]"
1,When siblings Judy and Peter discover an encha...,17.015539,Jumanji,"[When, siblings, Judy, and, Peter, discover, a...",[Jumanji]
2,A family wedding reignites the ancient feud be...,11.712900,Grumpier Old Men,"[A, family, wedding, reignites, the, ancient, ...","[Grumpier, Old, Men]"
3,"Cheated on, mistreated and stepped on, the wom...",3.859495,Waiting to Exhale,"[Cheated, on, ,, mistreated, and, stepped, on,...","[Waiting, to, Exhale]"
4,Just when George Banks has recovered from his ...,8.387519,Father of the Bride Part II,"[Just, when, George, Banks, has, recovered, fr...","[Father, of, the, Bride, Part, II]"
...,...,...,...,...,...
42190,A film archivist revisits the story of Rustin ...,0.386450,The Burkittsville 7,"[A, film, archivist, revisits, the, story, of,...","[The, Burkittsville, 7]"
42191,It's the year 3000 AD. The world's most danger...,0.661558,Caged Heat 3000,"[It, 's, the, year, 3000, AD, ., The, world, '...","[Caged, Heat, 3000]"
42192,An artist struggles to finish his work while a...,0.178241,Century of Birthing,"[An, artist, struggles, to, finish, his, work,...","[Century, of, Birthing]"
42193,"In a small town live two brothers, one a minis...",0.003503,Satan Triumphant,"[In, a, small, town, live, two, brothers, ,, o...","[Satan, Triumphant]"


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Reemplazar los valores nulos en la columna 'overview' con un valor predeterminado
df['overview'].fillna('Sin descripción', inplace=True)

# Crear una instancia del vectorizador de Bolsa de Palabras con límite máximo de características
vectorizer = CountVectorizer(max_features=100)

# Realizar la tokenización y vectorización de la columna 'overview'
overview_vectors = vectorizer.fit_transform(df['overview_tokens'].apply(lambda x: ' '.join(x)))

# Crear el DataFrame con los vectores de Bolsa de Palabras
overview_df = pd.DataFrame(overview_vectors.toarray(), columns=vectorizer.get_feature_names_out())

# Concatenar el DataFrame de características con el DataFrame original
df_with_features = pd.concat([df, overview_df], axis=1)


In [7]:
overview_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42195 entries, 0 to 42194
Data columns (total 100 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   about     42195 non-null  int64
 1   after     42195 non-null  int64
 2   all       42195 non-null  int64
 3   an        42195 non-null  int64
 4   and       42195 non-null  int64
 5   are       42195 non-null  int64
 6   as        42195 non-null  int64
 7   at        42195 non-null  int64
 8   back      42195 non-null  int64
 9   be        42195 non-null  int64
 10  been      42195 non-null  int64
 11  between   42195 non-null  int64
 12  but       42195 non-null  int64
 13  by        42195 non-null  int64
 14  can       42195 non-null  int64
 15  daughter  42195 non-null  int64
 16  day       42195 non-null  int64
 17  during    42195 non-null  int64
 18  family    42195 non-null  int64
 19  father    42195 non-null  int64
 20  film      42195 non-null  int64
 21  find      42195 non-null  int64
 2

In [8]:
df_with_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42195 entries, 0 to 42194
Columns: 105 entries, overview to young
dtypes: float64(1), int64(100), object(4)
memory usage: 33.8+ MB


In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud del coseno entre las películas
similarity_matrix = cosine_similarity(overview_df)



h:\Henry_bootcamp\PI_1\venv\Lib\site-packages\sklearn\utils\extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [14]:
similarity_matrix.shape


(42195, 42195)

In [15]:
similarity_matrix

array([[1.        , 0.4451741 , 0.48261709, ..., 0.48653363, 0.64936449,
        0.61487546],
       [0.4451741 , 1.        , 0.42355872, ..., 0.20333143, 0.56656831,
        0.51714781],
       [0.48261709, 0.42355872, 1.        , ..., 0.1028689 , 0.69973089,
        0.63702206],
       ...,
       [0.48653363, 0.20333143, 0.1028689 , ..., 1.        , 0.29139113,
        0.14744196],
       [0.64936449, 0.56656831, 0.69973089, ..., 0.29139113, 1.        ,
        0.82167269],
       [0.61487546, 0.51714781, 0.63702206, ..., 0.14744196, 0.82167269,
        1.        ]])

In [10]:
def obtener_recomendaciones(nombre_pelicula):
    # Obtener el índice de la película de referencia
    indice_pelicula = df[df['title'] == nombre_pelicula].index[0]
    
    # Obtener la fila correspondiente a la película de referencia en la matriz de similitud
    similitudes_pelicula = similarity_matrix[indice_pelicula]
    
    # Obtener los índices de las 5 películas más similares (excluyendo la película de referencia)
    indices_recomendadas = similitudes_pelicula.argsort()[-6:-1][::-1]
    
    # Crear una lista con los títulos de las películas recomendadas
    peliculas_recomendadas = []
    for indice in indices_recomendadas:
        titulo = df.loc[indice, 'title']
        peliculas_recomendadas.append(titulo)
    
    return peliculas_recomendadas

In [11]:
pelicula_referencia = "Toy Story"
recomendaciones = obtener_recomendaciones(pelicula_referencia)

print(recomendaciones)

['Ugetsu', 'Time Of My Life', "Thrashin'", 'Grave of the Fireflies', 'Rams']
